In [150]:
import pandas as pd
from PIL import Image,ImageDraw,ImageFont
import numpy as np

In [151]:
def setstring2list(setstring):
    tmp = setstring.lstrip("{").rstrip("}")
    tmp = tmp.split(",")
    tmp = [item.translate(None," '") for item in tmp]
    return tmp

In [152]:
limefeat_alphasort = pd.read_csv("limefeat_alphasort.csv")
df = pd.read_csv("lime-robustness-all-score-outcome.csv")
df.sort_values(by=['unit_score','officer_id','nft'],ascending=[False,True,True],inplace=True)
df.reset_index(drop=True,inplace=True)
df['unq'] = df.unq.apply(setstring2list)

# Subsetting LIME runs

In [153]:
officer_ids = list(df.officer_id.unique())
#nsmps = [50,100,500,1000,3000,5000,7000]
nsmps = [5000]
#nfts = [2,5,7,11,13,17]
nfts = [17]
outcomes = [0,1]

df = df[df.officer_id.isin(officer_ids)]
df = df[df.nsmp.isin(nsmps)]
df = df[df.nft.isin(nfts)]
df = df[df.outcome.isin(outcomes)]

df.reset_index(drop=True,inplace=True)

In [154]:
rowmod = 'smp-5000_'+str(nfts[0])

# Subsetting Features

In [155]:
rectype = ['arrests','arstat','demarrests','dispatch','fi','ic','ir','irag','ocag','ocnd','outemp','shifts','ts']
#rectype = ['arrests','dispatch','ocnd','outemp','ts']
#temp_aggs = ['1d', '1m', '1w', '1y', '5y', 'all']

limefeat_alphasort = limefeat_alphasort[limefeat_alphasort.record_type.isin(rectype)]
#limefeat_alphasort = limefeat_alphasort[limefeat_alphasort.temp_agg.isin(temp_aggs)]

limefeat_alphasort.reset_index(drop=True,inplace=True)

colmod = 'all'

# Collapse

In [156]:
collapse = True

In [157]:
if collapse==True:
    featlists = []
    for officer_id in df.officer_id.unique():
        featlist = list(df.unq[df.officer_id==officer_id])
        featlist = [item for sublist in featlist for item in sublist]
        featlist = list(set(featlist))
        featlists.append(featlist)

    df = df[['officer_id','unit_score','outcome']].groupby('officer_id').agg(np.mean)
    df.sort_values(by='unit_score',ascending=False,inplace=True)
    df['unq'] = featlists
    df['nft'] = None
    df['nsmp'] = None
    df.reset_index(inplace=True)

# Color Settings

In [158]:
cathue = {
    'ts':'0',
    'fi':'32',
    'ir':'60',
    'shifts':'115',
    'dispatch':'180',
    'arrests':'240',
    'ic':'270',
    'demarrests':'300',
    'ocnd':'200'
    }

bkgdcolor = 'hsl(0,0%,50%)'
linecolordark = 'hsl(0,0%,0%)'
linecolormed = 'hsl(0,0%,25%)'
linecolorlight = 'hsl(0,0%,33%)'
textcolor = 'hsl(0,0%,0%)'

# Plotter

In [159]:
rectwidth = 10
rectheight = 10
px_w = len(limefeat_alphasort) * rectwidth
px_h = len(df) * rectheight

In [160]:
def offtableart(df,limefeat_alphasort):
    
    canvas = Image.new('RGB',(px_w,px_h),bkgdcolor)
    officer_id = None
    nft = None
    
    for i in df.index:
        feats = df.unq.loc[i]
        ycoord = i * rectheight
        
        newid = df.officer_id.loc[i]
        newft = df.nft.loc[i]  
        
        for feat in feats:
            tmp = limefeat_alphasort[limefeat_alphasort.feature==feat]
            
            try:
                record_type = tmp.record_type.iloc[0]
                idx = tmp.index[0]
                
                xcoord = idx * rectwidth
                bbox = [xcoord, ycoord, xcoord + rectwidth, ycoord + rectheight]

                try:
                    hue = cathue[record_type]
                    sat = '100%'
                    val = '50%'
                except:
                    hue = '0'
                    sat = '0%'
                    val = '100%'
                    
                fillcolor = 'hsl('+hue+","+sat+","+val+")"
                draw = ImageDraw.Draw(canvas)
                draw.rectangle(bbox,fill=fillcolor,outline=None)
            
            except:
                pass     
        
        if newid!=officer_id:
            draw.line([(0,ycoord),(px_w,ycoord)],width=1,fill=linecolordark)
            officer_id = newid
            nft = newft
        elif newft!=nft:
            draw.line([(0,ycoord),(px_w,ycoord)],width=1,fill=linecolormed)
            nft = newft
        else:
            draw.line([(0,ycoord),(px_w,ycoord)],width=1,fill=linecolorlight)
    
    return canvas

In [161]:
im = offtableart(df,limefeat_alphasort)
w = im.width
h = im.height
x = 100
canvas = Image.new('RGB',((w+x),(h)),bkgdcolor)
canvas.paste(im,(x,0))
im = None

In [162]:
draw = ImageDraw.Draw(canvas)
for i in df.index:
    officer_id = df.officer_id.loc[i]
    nft = df.nft.loc[i]
    nsmp = df.nsmp.loc[i]
    
    text = str(officer_id)+","+str(nft)+","+str(nsmp)
    font = ImageFont.truetype('VeraMono.ttf', rectheight)
    
    xpos = 5
    ypos = i * rectheight
    draw.text((xpos,ypos),text=text,font=font,fill=textcolor)

In [163]:
w = canvas.width
h = canvas.height
y = 400
unicanvas = Image.new('RGB',((w),(h+y)),bkgdcolor)
unicanvas.paste(canvas,(0,y))
canvas = None

In [164]:
draw = ImageDraw.Draw(unicanvas)
for i in limefeat_alphasort.index:
    feature = limefeat_alphasort.featstr.loc[i]
    text = str(feature)
    
    font = ImageFont.truetype('VeraMono.ttf', rectwidth)
    fontWidth, fontHeight = font.getsize(text)
    
    imtmp = Image.new('RGBA', (fontWidth, fontHeight))
    drawtmp = ImageDraw.Draw(imtmp)
    drawtmp.text((0,0),text=text,font=font,fill=textcolor)
    imtmp = imtmp.rotate(90,expand=1)
    
    xpos = i * rectwidth + x - 2
    ypos = y - imtmp.height - 2
    unicanvas.paste(imtmp,(xpos,ypos),imtmp)
    
    linex = i * rectwidth + x
    liney = y
    draw.line([(linex,liney),(linex,unicanvas.height)],width=1,fill=linecolorlight)

In [165]:
unicanvas.save('/Users/damoncrockett/Desktop/Leimplatz/'+rowmod+'___'+colmod+'.png')